Load local model

In [1]:
# Local
from langchain_community.chat_models import ChatOllama
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser

from libs.core.tests.unit_tests.output_parsers.test_openai_tools import message

ds_chat = ChatOllama(model="deepseek-r1:7b")

C:\Users\guosh\AppData\Local\Temp\ipykernel_69060\862749496.py:6: LangChainDeprecationWarning: The class `ChatOllama` was deprecated in LangChain 0.3.1 and will be removed in 1.0.0. An updated version of the class exists in the :class:`~langchain-ollama package and should be used instead. To use it run `pip install -U :class:`~langchain-ollama` and import as `from :class:`~langchain_ollama import ChatOllama``.
  ds_chat = ChatOllama(model="deepseek-r1:7b")


Maintain message list

In [7]:
from langchain_core.prompts import MessagesPlaceholder
from langchain_core.messages import SystemMessage, HumanMessage, AIMessage
prompt = ChatPromptTemplate.from_messages(
    [
        SystemMessage(
            content="You are a helpful assistant. Answer all questions to the best of your ability. Answer in English.",
        ),
        MessagesPlaceholder(variable_name="messages"),
    ]
)
chain = prompt | ds_chat
mes_list=    {
        "messages": [
            HumanMessage(
                content="Translate from English to French: I love programming."
            ),
            AIMessage(content="J'adore la programmation."),
            HumanMessage(content="What did you just say?"),
        ],
    }
ai_msg = chain.invoke(mes_list)
print(ai_msg.content)

<think>
Okay, so the user initially asked me to translate "I love programming." into French. My response was straightforward with "J'adore la programmation." Now, looking at the conversation history, after that translation, the user followed up by asking, "What did you just say?" 

Hmm, I think they're testing my ability or maybe curious about how I handled the request. They might be checking if I understood correctly when translating from English to French.

I need to make sure my response is clear and helpful here. Maybe acknowledge that I translated it for them. Adding an emoji could make the response friendlier, which sometimes helps in building rapport with users.

Also, offering further assistance shows that I'm ready to help more if they have another question or need something else. So putting it all together: a friendly acknowledgment followed by an offer to continue assisting.
</think>

Je viens de te répondre que je t'adore ! C'est donc un cauchemar ? Je me demande si tu veux